In [1]:
import time
from copy import deepcopy

import numpy as np
import pandas as pd
import rectools
from rectools import Columns
from rectools.dataset import Dataset
from rectools.dataset import Interactions
from rectools.metrics import (
    Precision,
    Recall,
    MAP,
    NDCG,
    calc_metrics,
    Serendipity,
    MeanInvUserFreq
)
from rectools.model_selection import TimeRangeSplitter
from rectools.models import RandomModel, PopularModel
from tqdm import tqdm

In [2]:
df_interactions = pd.read_csv('../data/interactions.csv')
df_interactions.rename(
    columns={
        "last_watch_dt": rectools.Columns.Datetime,
        "total_dur": rectools.Columns.Weight},
    inplace=True
)

interactions = Interactions(df_interactions)

items = pd.read_csv('../data/items.csv')

In [3]:
class RecoEvaluation:
    def __init__(self, models, metrics, splitter, k):
        self.models = models
        self.metrics = metrics
        self.splitter = splitter
        self.k = k

    def evaluate(self, interactions):
        n_splits = self.splitter.n_splits
        results = []

        fold_iterator = self.splitter.split(interactions, collect_fold_stats=True)

        for train_ids, test_ids, fold_info in tqdm(fold_iterator, total=n_splits):
            print(f"Fold {fold_info['i_split']}")
            print(fold_info)

            df_train = interactions.df.iloc[train_ids]
            dataset = Dataset.construct(df_train)

            df_test = interactions.df.iloc[test_ids][Columns.UserItem]
            test_users = np.unique(df_test[Columns.User])

            catalog = df_train[Columns.Item].unique()

            for model_name, model in self.models.items():
                model_copy = deepcopy(model)  # Ensure a deep copy of the model
                start = time.time()
                model_copy.fit(dataset)
                end = time.time()
                recos = model_copy.recommend(
                    users=test_users,
                    dataset=dataset,
                    k=self.k,
                    filter_viewed=True,
                )

                metric_values = calc_metrics(
                    self.metrics,
                    reco=recos,
                    interactions=df_test,
                    prev_interactions=df_train,
                    catalog=catalog,
                )

                res = {"fold": fold_info["i_split"], "model": model_name, "time": end - start}
                res.update(metric_values)
                results.append(res)

        return pd.DataFrame(results)

In [4]:
SEED = 42
N_SPLITS = 3
K = 10

models = {
    'RandomModel': RandomModel(random_state=SEED),
    'PopularModel': PopularModel()
}

metrics = {
    'Precision@1': Precision(k=1),
    'Precision@5': Precision(k=5),
    'Precision@10': Precision(k=10),
    'Recall@1': Recall(k=1),
    'Recall@5': Recall(k=5),
    'Recall@10': Recall(k=10),
    'MAP@1': MAP(k=1, divide_by_k=False),
    'MAP@5': MAP(k=5, divide_by_k=False),
    'MAP@10': MAP(k=10, divide_by_k=False),
    'NDCG@1': NDCG(k=1, log_base=3),
    'NDCG@5': NDCG(k=5, log_base=3),
    'NDCG@10': NDCG(k=10, log_base=3),
    'MIUF@1': MeanInvUserFreq(k=1),
    'MIUF@5': MeanInvUserFreq(k=5),
    'MIUF@10': MeanInvUserFreq(k=10),
    'Serendipity@1': Serendipity(k=1),
    'Serendipity@5': Serendipity(k=5),
    'Serendipity@10': Serendipity(k=10)
}

splitter = TimeRangeSplitter(
    "7D",
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

In [5]:
evaluator = RecoEvaluation(models, metrics, splitter, K)
results_df = evaluator.evaluate(interactions)

  0%|          | 0/3 [00:00<?, ?it/s]

Fold 0
{'i_split': 0, 'start': Timestamp('2021-08-02 00:00:00', freq='7D'), 'end': Timestamp('2021-08-09 00:00:00', freq='7D'), 'train': 4266013, 'train_users': 797423, 'train_items': 15237, 'test': 263681, 'test_users': 98184, 'test_items': 6602}


 33%|███▎      | 1/3 [00:17<00:34, 17.07s/it]

Fold 1
{'i_split': 1, 'start': Timestamp('2021-08-09 00:00:00', freq='7D'), 'end': Timestamp('2021-08-16 00:00:00', freq='7D'), 'train': 4649162, 'train_users': 850489, 'train_items': 15415, 'test': 279422, 'test_users': 103511, 'test_items': 6698}


 67%|██████▋   | 2/3 [00:35<00:17, 17.67s/it]

Fold 2
{'i_split': 2, 'start': Timestamp('2021-08-16 00:00:00', freq='7D'), 'end': Timestamp('2021-08-23 00:00:00', freq='7D'), 'train': 5051815, 'train_users': 906071, 'train_items': 15577, 'test': 298878, 'test_users': 110076, 'test_items': 6679}


100%|██████████| 3/3 [00:54<00:00, 18.06s/it]


In [6]:
results_df.groupby("model").mean()

,fold,time,Precision@1,Recall@1,Precision@5,Recall@5,Precision@10,Recall@10,NDCG@1,NDCG@5,NDCG@10,MAP@1,MAP@5,MAP@10,MIUF@1,MIUF@5,MIUF@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,,,,,,,,,,,,
PopularModel,1.0,1.381567,0.076432,0.042720,0.052402,0.137413,0.033903,0.173492,0.076432,0.057932,0.043084,0.042720,0.078295,0.084109,2.377055,3.066979,3.713390,0.000002,0.000003,0.000002
RandomModel,1.0,0.000026,0.000169,0.000069,0.000176,0.000345,0.000176,0.000687,0.000169,0.000173,0.000175,0.000069,0.000155,0.000200,15.616147,15.611575,15.610409,0.000004,0.000006,0.000006


In [8]:
pivot_results = pd.DataFrame(results_df).drop("fold", axis=1).groupby(["model"], sort=False).agg(
    ["mean"])
mean_metric_subset = [(metric, agg) for metric, agg in pivot_results.columns if agg == 'mean']
(
    pivot_results.style
    .highlight_min(subset=mean_metric_subset, color='lightcoral', axis=0)
    .highlight_max(subset=mean_metric_subset, color='lightgreen', axis=0)
)

,time,Precision@1,Recall@1,Precision@5,Recall@5,Precision@10,Recall@10,NDCG@1,NDCG@5,NDCG@10,MAP@1,MAP@5,MAP@10,MIUF@1,MIUF@5,MIUF@10,Serendipity@1,Serendipity@5,Serendipity@10
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
model,,,,,,,,,,,,,,,,,,,
RandomModel,0.000026,0.000169,0.000069,0.000176,0.000345,0.000176,0.000687,0.000169,0.000173,0.000175,0.000069,0.000155,0.000200,15.616147,15.611575,15.610409,0.000004,0.000006,0.000006
PopularModel,1.381567,0.076432,0.042720,0.052402,0.137413,0.033903,0.173492,0.076432,0.057932,0.043084,0.042720,0.078295,0.084109,2.377055,3.066979,3.713390,0.000002,0.000003,0.000002


In [68]:
class RecoVisualisation:
    def __init__(self, model, dataset, user_ids, item_data):
        self.model = model
        self.dataset = dataset
        self.user_ids = user_ids
        self.item_data = item_data

    def visualise(self):
        for user_id in self.user_ids:
            user_history = self.dataset.interactions.df.loc[
                self.dataset.interactions.df.user_id == user_id]
            user_recos = self.model.recommend(
                users=[user_id],
                dataset=self.dataset,
                k=K,
                filter_viewed=True
            )
            user_history = user_history.copy()
            user_recos = user_recos.copy()

            user_history['view_cnt'] = user_history['item_id'].map(
                self.dataset.interactions.df['item_id'].value_counts())

            user_recos['view_cnt'] = user_recos['item_id'].map(
                self.dataset.interactions.df['item_id'].value_counts())

            columns_show = list(set(list(self.item_data.columns) + ['item_id', 'view_cnt']))

            print(f"\nUser ID {user_id}")
            if not user_history.empty:
                user_history = user_history.merge(
                    self.item_data,
                    left_on='item_id',
                    right_on='item_id',
                ).sort_values(by='datetime').tail(10)
                print(f"\nUser ID {user_id} - Watched items:")
                display(user_history[columns_show])

            if not user_recos.empty:
                user_recos = user_recos.merge(
                    self.item_data,
                    left_on='item_id',
                    right_on='item_id',
                )
                print(f"\nUser ID {user_id} - Recommended items:")
                display(user_recos[columns_show])

In [69]:
user_ids = [666262, 672861, 955527]

model = RandomModel(random_state=42)
dataset = Dataset.construct(df_interactions)
model.fit(dataset)

reco_viz = RecoVisualisation(model, dataset, user_ids,
                             items[['item_id', 'title', 'genres']])

In [70]:
reco_viz.visualise()


User ID 666262

User ID 666262 - Watched items:


,title,genres,item_id,view_cnt
0,Дом ночных призраков,"зарубежные, криминал, детективы, ужасы",93,68581



User ID 666262 - Recommended items:


,title,genres,item_id,view_cnt
0,Ода радости,комедии,7419,6
1,Последняя битва,"драмы, военные",9109,19
2,Преисподняя,"драмы, детективы, триллеры, вестерн",13917,2
3,Лихорадка,ужасы,13332,4
4,Вечность,драмы,1331,180
5,Леший,"триллеры, криминал, детективы",15448,1
6,Богема,"драмы, мюзиклы, мелодрамы",1123,896
7,Шедевр,"драмы, комедии",9933,3
8,Единоборства для детей (3-6 лет) Школа героев ...,"единоборства, фитнес, для детей",3287,728
9,Американский пирог: Все в сборе,комедии,8478,3



User ID 672861

User ID 672861 - Watched items:


...

,title,genres,item_id,view_cnt
0,Ода радости,комедии,7419,6
1,Последняя битва,"драмы, военные",9109,19
2,Преисподняя,"драмы, детективы, триллеры, вестерн",13917,2
3,Лихорадка,ужасы,13332,4
4,Вечность,драмы,1331,180
5,Леший,"триллеры, криминал, детективы",15448,1
6,Богема,"драмы, мюзиклы, мелодрамы",1123,896
7,Шедевр,"драмы, комедии",9933,3
8,Единоборства для детей (3-6 лет) Школа героев ...,"единоборства, фитнес, для детей",3287,728
9,Американский пирог: Все в сборе,комедии,8478,3



User ID 955527

User ID 955527 - Watched items:


,title,genres,item_id,view_cnt
0,Признание 5,для взрослых,21,193123



User ID 955527 - Recommended items:


,title,genres,item_id,view_cnt
0,Ода радости,комедии,7419,6
1,Последняя битва,"драмы, военные",9109,19
2,Преисподняя,"драмы, детективы, триллеры, вестерн",13917,2
3,Лихорадка,ужасы,13332,4
4,Вечность,драмы,1331,180
5,Леший,"триллеры, криминал, детективы",15448,1
6,Богема,"драмы, мюзиклы, мелодрамы",1123,896
7,Шедевр,"драмы, комедии",9933,3
8,Единоборства для детей (3-6 лет) Школа героев ...,"единоборства, фитнес, для детей",3287,728
9,Американский пирог: Все в сборе,комедии,8478,3
